## Master's Thesis - Machine Learning in Asset Pricing

### Thomas Theodor Kjølbye 

The Following script handles the first part of the data processing. On my computer, the entire script takes approximately 10-15 minutes to run. The data consist of individual firm characteristics as well as macroeconomic variables and are generously made available by Professors Gu, Kelly, Xiu, and Goyal. 

In [2]:
import os
import csv
import pandas as pd
import numpy as np
import datetime as dt
import SupportFunctions as supp

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Initialize warning log container 
log = list()

In [3]:
# Load macroeconomic predictors 
macro_data_raw = pd.read_csv(os.path.dirname(os.getcwd()) + '\\PredictorData2021 - Monthly.csv')

# Because of the 1000-separator (,) index (price) is loaded as an object instead of a float. Check with df.dtypes
macro_data_raw["Index"].replace(',', '', inplace = True, regex = True)
macro_data_raw["Index"] = macro_data_raw["Index"].astype({"Index": float}) 

# Initialize empty dataframe
column_names = ['date', 'dp_macro', 'ep_macro', 'bm_macro', 'ntis', 'tbl', 'tms', 'dfy', 'svar']
macro_data = pd.DataFrame(columns = column_names, index = range(macro_data_raw.shape[0]))

# Create macroeconomic predictors leveraged in Gu, Kelly, and Xiu (2020)
macro_data['date'] = macro_data_raw["yyyymm"]
macro_data['tbl'] = macro_data_raw["tbl"]
macro_data['svar'] = macro_data_raw["svar"]
macro_data['dp_macro'] = abs(np.log(macro_data_raw['D12']) - np.log(macro_data_raw['Index']))
macro_data['ep_macro'] = abs(np.log(macro_data_raw['E12']) - np.log(macro_data_raw['Index']))
macro_data['bm_macro'] = macro_data_raw["b/m"]
macro_data["ntis"] = macro_data_raw["ntis"]
macro_data["tms"] = macro_data_raw["lty"] - macro_data_raw["tbl"]
macro_data["dfy"] = macro_data_raw["BAA"] - macro_data_raw["AAA"]

# Add constant for interaction terms and truncate at first and last returns observation
macro_data = macro_data[(macro_data["date"] > 195612) & (macro_data["date"] < 201701)] # Date of first and last returns observation
macro_data["constant"] = 1 # Add column of ones for interaction terms later
#macro_data.date = pd.to_datetime(macro_data.date, format = '%Y%m', errors='coerce')


# Export macro_data - Save outside of wd (repo)
#macro_data.to_csv(os.path.dirname(os.getcwd()) + "\\macro_data.csv", index = False)

In [4]:
# Load firm characteristics
firm_data_raw = pd.read_csv(os.path.dirname(os.getcwd()) + '\\datashare.zip')
firm_data_raw.columns = firm_data_raw.columns.str.lower()
firm_data_raw["date"] = firm_data_raw["date"].floordiv(100) #https://stackoverflow.com/questions/33034559/how-to-remove-last-the-two-digits-in-a-column-that-is-of-integer-type
#firm_data_raw.date = pd.to_datetime(firm_data_raw.date, format = '%Y%m%d', errors = 'coerce')
#print("The firm characteristics dataset is {:1.3f} GB".format(firm_data_raw.memory_usage().sum()/(1024 ** 3)))

# Downcast from 64bit float to 32bit
supp.downcast(firm_data_raw)

Before downcast: 2.976 GB and float64    95
int64       2
dtype: int64
After downcast: 1.488 GB and float32    95
int32       2
dtype: int64


In [5]:
# Load raw returns data
returns_raw = pd.read_csv(os.path.dirname(os.getcwd()) + '\\RET.txt')
delist_ret = pd.read_csv(os.path.dirname(os.getcwd()) + '\\delist_ret.txt', sep = "\t")
delist_name = pd.read_csv(os.path.dirname(os.getcwd()) + '\\delist_nama.txt', sep = "\t")

# Clean
returns_raw.columns = returns_raw.columns.str.strip()
delist_ret.columns = delist_ret.columns.str.strip()
delist_name.columns = delist_name.columns.str.strip()

# Clean
returns_raw.columns = returns_raw.columns.str.lower()
delist_ret.columns = delist_ret.columns.str.lower()
delist_name.columns = delist_name.columns.str.lower()

# Clean
delist_name.rename(columns = {"kypermno":"permno"}, inplace = True)
delist_ret.rename(columns = {"kypermno":"permno"}, inplace = True)
returns_raw.rename(columns = {"col1":"date"}, inplace = True)

# Clean
returns_raw.drop(["col0", "altdt"], axis = 1, inplace = True)

# Clean
returns_raw["date"] = returns_raw["date"].floordiv(100) 
#returns_raw.date = pd.to_datetime(returns_raw.date, format = '%Y%m%d', errors = 'coerce')
#delist_name.namedt = pd.to_datetime(delist_name.namedt, format = '%d-%m-%Y', errors = 'coerce')
#delist_name.nameenddt = pd.to_datetime(delist_name.nameenddt, format = '%d-%m-%Y', errors = 'coerce')
#delist_ret.dlstdt = pd.to_datetime(delist_ret.dlstdt, format = '%d-%m-%Y', errors = 'coerce')

# Clean - not for deilst_name (done later) 
returns_raw.set_index("permno", inplace = True)
delist_ret.set_index("permno", inplace = True)

# Start and end date of data period
start_date = 195701
end_date = 201612
#start_date = pd.datetime(1957, 1, 1)
#end_date = pd.datetime(2016, 12, 30)

In [6]:
# Returns data continued: Compute first and last date of each permno
first_date_permno = delist_name.sort_values("namedt", ascending = True).groupby("permno").head(1).drop("nameenddt", axis = 1)
last_date_permno = delist_name.sort_values("nameenddt", ascending = True).groupby("permno").tail(1).drop("namedt", axis = 1)

# Set index for pd.concat
first_date_permno.set_index("permno", inplace = True)
last_date_permno.set_index("permno", inplace = True)

# Combine with pd.concat
first_last_date_permno = pd.concat([first_date_permno, last_date_permno], axis = 1)

In [7]:
# Returns data continues: Merge to raw returns data 
returns = returns_raw.join(first_last_date_permno, on = "permno", how = "inner")
returns = returns.join(delist_ret, on = "permno", how = "inner")

In [8]:
# Clean returns
returns.rename(columns = {"sh":"shrcd", "sic":"siccd", 
                                      "prcprev":"altprc", "ex":"exchcd", 
                                      "shr":"shrout"}, inplace = True)

returns = returns[(returns.shrcd == 10) | (returns.shrcd == 11)]
returns = returns[(returns.date >= start_date) & (returns.date <= end_date)]
returns = returns.drop(["shrcd", "namedt", "nameenddt", "dlstdt", "exchcd", "siccd", "altprc", "shrout", "dlstcd", "dlret"], axis = 1)
returns = returns.reset_index()

In [9]:
# Remove missing observations according to Gu, Kelly, and Xiu (2020)
returns = returns[returns["ret"] > -10]

In [10]:
# Filter out rows missing in returns data by merging
data = firm_data_raw.merge(returns, on = ["permno", "date"], how = "inner")

In [11]:
# Merge data with macroeconomic predictors 
data = data.merge(macro_data, on = "date")
data = data.set_index(["permno", "date"]) # 2261428 x 105 (94 char, industry dummy, returns, 8 macro, ones)

In [12]:
data

,,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,mom6m,mom12m,...,ret,dp_macro,ep_macro,bm_macro,ntis,tbl,tms,dfy,svar,constant
permno,date,,,,,,,,,,,,,,,,,,,,,
10006,195701,8.224900e+04,1.122846,1.260784,0.047180,9.569954,0.025742,0.046433,0.044843,-0.059517,-0.120199,...,0.064378,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
10014,195701,3.903375e+03,0.426734,0.182102,-0.275641,6.237836,0.072103,0.046433,-0.086957,-0.115385,-0.041667,...,0.095238,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
10022,195701,9.273250e+03,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046433,-0.060377,-0.039550,-0.108751,...,0.102041,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
10030,195701,5.446588e+04,0.926038,0.857547,0.018171,9.825336,0.021700,0.046433,0.044633,0.050470,0.133650,...,-0.047091,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
10057,195701,4.025000e+04,1.247748,1.556875,0.025785,7.901007,0.025506,0.046433,0.086667,0.055247,0.182844,...,-0.090062,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90487,201611,1.548740e+05,0.122542,0.015017,0.318973,10.737428,0.027743,0.108412,0.066225,0.275889,0.328727,...,0.130849,3.878496,3.165980,0.303286,-0.027452,0.0045,0.0222,0.0085,0.000946,1
91096,201611,7.170900e+04,2.020000,4.080399,0.960797,12.006117,0.061115,-0.000388,0.014851,0.355705,-0.437326,...,0.048781,3.878496,3.165980,0.303286,-0.027452,0.0045,0.0222,0.0085,0.000946,1
92054,201611,9.571450e+03,1.670872,2.791814,0.167215,11.664936,0.104895,-0.007036,-0.148148,-0.571428,-0.778612,...,-0.808696,3.878496,3.165980,0.303286,-0.027452,0.0045,0.0222,0.0085,0.000946,1


In [13]:
# Separate dataset in 1) firm chars + macro (FM), 2) returns, and 3) industry codes

# 1)
FM_todrop = ["ret", "sic2"]
FM_data = data.drop(FM_todrop, axis = 1).reset_index()

# 2) 
returns = data.ret.reset_index()

# 3) 
industry_code = data.sic2.reset_index()

In [14]:
# Save data
FM_data.to_csv(os.path.dirname(os.getcwd()) + '\\FM2_data.csv', header = True, index = False)
returns.to_csv(os.path.dirname(os.getcwd()) + '\\returns2_data.csv', header = True, index = False)
industry_code.to_csv(os.path.dirname(os.getcwd()) + '\\industry2_codes.csv', header = True, index = False)

In [17]:
# Stats
FM_data.permno.unique().shape # 29906 unique firms
avg = [x.shape[0] for x in FM_data.groupby("date").permno.unique()]
sum(avg)/len(avg) # 5222.66 firms on average at each point in time (month)